## Experiments: Sensitive Analysis Hyperparameter RSAST:


It is runned RSAST in a set of UCR datasets with a predefined number of runs ("runs"). Then, it is selected a range ("range_total") between 1, 10, 30 ,50 and 100 for the selected dataset.

In [91]:
#configure directory to import sast libraries
import sys 
import os 
#add sast library path
file_path = os.path.expanduser("~/random_sast/sast")
sys.path.append(file_path)
#add cd_diagram library path
file_path = os.path.expanduser("~/random_sast/cd_diagram")
sys.path.append(file_path)

#sys.path

In [92]:
from sast import *
from sktime.datasets import load_UCR_UEA_dataset, tsc_dataset_names
from sktime.classification.kernel_based import RocketClassifier
import time
import pandas as pd
import os

### Reading Datasets

In [93]:
# Set directory where the csv files are located
directory = os.getcwd()+'/ResultsByClassifier'

# Create an empty list to store the dataframes
dfs = []

# Loop through all files in the directory
for filename in os.listdir(directory):
    # Check if the file starts with "df_overall_tunning" and ends with ".csv"
    if filename.endswith(".csv"):
        # Read the csv file into a dataframe
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath)
        # Append the dataframe to the list
        df['filename']=filename
        dfs.append(df)

# Concatenate all the dataframes in the list into one dataframe
df_other_methods = pd.concat(dfs, ignore_index=True)
df_other_methods=df_other_methods[['filename','folds:','0']]
df_other_methods['method']=df_other_methods['filename'].str.split('_').str[0]
df_other_methods
df_other_methods=df_other_methods.rename(columns={'folds:':'dataset','0':'score'})
df_other_methods=df_other_methods[['dataset','score','method']]


In [94]:
df_other_methods['method'].unique()

array(['SAST', 'ROCKET', 'HIVE-COTEv1', 'BOSS', 'RISE', 'TSF',
       'InceptionTime', 'TS-CHIEF', 'S-BOSS', 'STC', 'ResNet',
       'ProximityForest', 'WEASEL', 'Catch22', 'cBOSS'], dtype=object)

In [95]:
# Set directory where the csv files are located
directory = os.getcwd()+'/ResultsRsast'
#directory = os.getcwd()+'/results_accuracy_per_ds'

# Create an empty list to store the dataframes
dfs = []

# Loop through all files in the directory
for filename in os.listdir(directory):
    # Check if the file starts with "df_overall_tunning" and ends with ".csv"
    if filename.startswith("df_overall_tunning") and filename.endswith(".csv"):
        # Read the csv file into a dataframe
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath)
        df['rpoint']=df['rpoint'].astype(str)
        df['nb_per_class']=df['nb_per_class'].astype(str)

        
        # Append the dataframe to the list
        dfs.append(df)

# Concatenate all the dataframes in the list into one dataframe
df_result = pd.concat(dfs, ignore_index=True)
df_result.head(3)


,Unnamed: 0,accuracy,time,cweights_time,fsubsequence_time,tdataset_time,tclassifier_time,dataset_name,classifier_name,rpoint,nb_per_class,method,len_method
0,0,0.573333,0.018448,0.004667,0.003236,0.006212,0.003818,SmoothSubspace,ACF&PACF: n_random_points=1 nb_inst_per_class=1,1,1,Rsast,ACF&PACF
1,1,0.566667,0.012903,0.005363,0.001630,0.001871,0.003664,SmoothSubspace,ACF: n_random_points=1 nb_inst_per_class=1,1,1,Rsast,ACF
2,2,0.613333,0.019624,0.006083,0.003176,0.005806,0.004071,SmoothSubspace,PACF: n_random_points=1 nb_inst_per_class=1,1,1,Rsast,PACF


In [96]:
#df_result=pd.read_csv("df_overall_tunning_SmoothSubspace3.csv")

df_result['dataset_name'].unique()


array(['SmoothSubspace', 'Chinatown', 'SonyAIBORobotSurface2',
       'DistalPhalanxOutlineCorrect', 'GunPoint', 'ItalyPowerDemand',
       'DistalPhalanxOutlineAgeGroup', 'ShapeletSim', 'SyntheticControl',
       'SonyAIBORobotSurface1', 'Coffee', 'BirdChicken'], dtype=object)

In [97]:
df_result.columns.unique()

Index(['Unnamed: 0', 'accuracy', 'time', 'cweights_time', 'fsubsequence_time',
       'tdataset_time', 'tclassifier_time', 'dataset_name', 'classifier_name',
       'rpoint', 'nb_per_class', 'method', 'len_method'],
      dtype='object')

### Hyperparameter Tunning RSAST 

#### Accuracy: Subsequence Lenght Method 

In [98]:

filter_param=df_result[df_result.rpoint=='(lenthg ts)//2']
filter_param.rpoint.unique()

array(['(lenthg ts)//2'], dtype=object)

In [99]:

filter_param=filter_param[filter_param.nb_per_class=='(max instances per class)//2']
filter_param.nb_per_class.unique()

array(['(max instances per class)//2'], dtype=object)

In [100]:
# create a pivot table with the mean of score by dataset

len_method_vs_ds=pd.pivot_table(filter_param, values='accuracy', index=['len_method'],columns=['dataset_name'], aggfunc='mean')
len_method_vs_ds=np.transpose(len_method_vs_ds)#[['ACF','PACF']]
len_method_vs_ds=len_method_vs_ds.reset_index()
df_rocket=df_other_methods[df_other_methods["method"]=="ROCKET"]
merged_df = len_method_vs_ds.merge(df_rocket,left_on='dataset_name', right_on='dataset',  how='left')
merged_df=merged_df.drop('method',axis=1)
merged_df

,dataset_name,ACF,ACF&PACF,Max ACF,Max PACF,None,PACF,dataset,score
0,Chinatown,0.956268,0.973178,0.961322,0.972400,0.961127,0.968902,Chinatown,0.982507
1,Coffee,0.964286,1.000000,0.964286,0.935714,0.971429,1.000000,Coffee,1.000000
2,DistalPhalanxOutlineCorrect,0.664493,0.710145,0.668841,0.691304,0.730435,0.709420,DistalPhalanxOutlineCorrect,0.771739
3,GunPoint,0.825778,0.979556,0.831111,0.967111,0.955111,0.972444,GunPoint,1.000000
4,ItalyPowerDemand,0.945384,0.945578,0.946615,0.945060,0.947716,0.944606,ItalyPowerDemand,0.968902
5,SmoothSubspace,0.825333,0.888444,0.810667,0.874222,0.825778,0.884444,SmoothSubspace,0.980000
6,SonyAIBORobotSurface2,0.756278,0.898496,0.760895,0.861630,0.822176,0.898566,SonyAIBORobotSurface2,0.916055


In [101]:
len_method_vs_ds=pd.pivot_table(filter_param, values='accuracy', index=['len_method'],columns=['dataset_name'], aggfunc='var')
len_method_vs_ds=np.transpose(len_method_vs_ds)#[['ACF','PACF']]
len_method_vs_ds=len_method_vs_ds.reset_index()
len_method_vs_ds

len_method,dataset_name,ACF,ACF&PACF,Max ACF,Max PACF,None,PACF
0,Chinatown,0.000210,0.000105,0.000194,0.000075,0.000252,0.000106
1,Coffee,0.000000,0.000000,0.001276,0.000255,0.000893,0.000000
2,DistalPhalanxOutlineCorrect,0.001041,0.000033,0.000148,0.000679,0.000306,0.000016
3,GunPoint,0.000583,0.000079,0.000595,0.000105,0.000301,0.000145
4,ItalyPowerDemand,0.000092,0.000018,0.000117,0.000122,0.000111,0.000049
5,SmoothSubspace,0.001093,0.000251,0.001726,0.001440,0.001028,0.000849
6,SonyAIBORobotSurface2,0.003548,0.000091,0.002278,0.000343,0.001508,0.000186


In [102]:
# create a pivot table with the mean of score by number of random points
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['rpoint'], aggfunc='mean')

rpoint,(lenthg ts)//2,1,10,100,30,50
len_method,,,,,,
ACF,0.856425,0.689619,0.758784,0.769028,0.753627,0.766047
ACF&PACF,0.922882,0.769391,0.793226,0.801660,0.794907,0.802196
Max ACF,0.855296,0.679644,0.743158,0.751406,0.727599,0.758477
Max PACF,0.909127,0.698627,0.764967,0.766697,0.748808,0.765181
None,0.892721,0.698144,0.744397,0.744392,0.734515,0.742097
PACF,0.921838,0.772806,0.787316,0.806516,0.804265,0.799760


In [103]:
# create a pivot table with the mean of score by number of instances per class
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
len_method,,,,,,
ACF,0.850791,0.687244,0.745676,0.773844,0.763623,0.771840
ACF&PACF,0.924333,0.760339,0.782520,0.808004,0.803532,0.805667
Max ACF,0.853637,0.646877,0.744265,0.757744,0.748818,0.764088
Max PACF,0.910880,0.682773,0.767170,0.767782,0.756877,0.768083
None,0.898420,0.664842,0.742530,0.757111,0.740363,0.753518
PACF,0.923618,0.764288,0.784552,0.803767,0.810992,0.805445


#### Time: Subsequence Lenght Method 

In [104]:
# create a pivot table with the mean of the time by dataset
pd.pivot_table(df_result, values='time', index=['len_method'],columns=['dataset_name'], aggfunc='mean')

dataset_name,BirdChicken,Chinatown,Coffee,DistalPhalanxOutlineAgeGroup,DistalPhalanxOutlineCorrect,GunPoint,ItalyPowerDemand,ShapeletSim,SmoothSubspace,SonyAIBORobotSurface1,SonyAIBORobotSurface2,SyntheticControl
len_method,,,,,,,,,,,,
ACF,81.630738,0.120657,43.827708,132.824072,231.141162,42.810175,0.674857,7.645123,1.599955,0.419210,0.815245,128.598058
ACF&PACF,297.280662,0.368780,148.320089,419.191777,679.493913,95.588660,1.649303,222.211859,2.648634,2.247746,3.008572,402.314136
Max ACF,2.745482,0.122675,3.134972,49.805655,76.582509,4.674037,0.681721,2.640422,1.595359,0.402863,0.627533,45.391469
Max PACF,107.885490,0.134552,6.470813,29.798286,46.293955,4.200933,0.747006,124.217464,1.849311,0.405555,0.495278,29.918500
None,1.853694,0.109866,1.756151,28.495936,43.753207,2.811864,0.665972,1.836391,1.571106,0.300002,0.397229,28.232846
PACF,208.928276,0.290148,105.678870,293.491555,452.581422,52.941237,1.651981,215.296044,2.606331,1.952652,2.348842,260.544206


In [105]:
# create a pivot table with the mean of the time by number of instances per class
pd.pivot_table(df_result, values='time', index=['len_method'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
len_method,,,,,,
ACF,48.643938,2.408007,23.829845,100.958694,49.818356,69.737185
ACF&PACF,140.594888,9.879895,94.304780,325.042851,175.067886,231.647165
Max ACF,14.091883,0.583890,4.976424,31.122473,13.260925,20.218350
Max PACF,9.053225,2.596105,25.310661,40.632923,34.174947,34.688503
None,7.794670,0.412087,3.132432,18.539203,8.092722,12.088692
PACF,92.621959,7.139428,70.762800,225.220860,122.114742,162.517941


In [106]:
# create a pivot table with the mean of the time by number of random points
pd.pivot_table(df_result, values='time', index=['len_method'],columns=['rpoint'], aggfunc='mean')

rpoint,(lenthg ts)//2,1,10,100,30,50
len_method,,,,,,
ACF,41.069520,1.495594,13.517231,132.225404,40.012599,66.387092
ACF&PACF,116.078250,22.368051,58.050266,414.248330,139.032105,224.531677
Max ACF,9.938760,0.509950,3.993417,38.539330,11.560758,19.334171
Max PACF,6.561226,17.899997,19.547818,43.344055,27.004143,31.872580
None,5.613996,0.324371,2.420662,23.178152,6.948198,11.376183
PACF,75.480858,20.715446,44.152286,283.848350,101.140794,153.481713


#### Variance: Subsequence Lenght Method 

In [107]:
# create a pivot table with the variance of the accuracy by number of instances per class
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['nb_per_class'], aggfunc='var')

nb_per_class,(max instances per class)//2,1,10,100,30,50
len_method,,,,,,
ACF,0.009765,0.041564,0.047788,0.041621,0.042785,0.042148
ACF&PACF,0.004775,0.044409,0.050636,0.044153,0.045366,0.046713
Max ACF,0.009815,0.049362,0.047050,0.047231,0.050320,0.044224
Max PACF,0.005239,0.058606,0.048741,0.052415,0.056022,0.053168
None,0.006296,0.053573,0.053066,0.050512,0.056206,0.050200
PACF,0.004490,0.040441,0.050086,0.046411,0.039660,0.045404


In [108]:
# create a pivot table with the variance of the accuracy by number of random points
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['rpoint'], aggfunc='var')

rpoint,(lenthg ts)//2,1,10,100,30,50
len_method,,,,,,
ACF,0.009716,0.049859,0.043517,0.038799,0.043759,0.039786
ACF&PACF,0.005371,0.050972,0.046168,0.044011,0.046321,0.044568
Max ACF,0.010301,0.050720,0.047230,0.043598,0.057696,0.043568
Max PACF,0.005886,0.063021,0.049566,0.049030,0.059927,0.049477
None,0.007613,0.061578,0.051401,0.048404,0.055762,0.051166
PACF,0.005119,0.044842,0.050571,0.039992,0.042282,0.044863


#### Another Results

In [109]:
df_result[['dataset_name','len_method','classifier_name']].describe()

,dataset_name,len_method,classifier_name
count,6630,6630,6630
unique,12,6,216
top,SmoothSubspace,ACF&PACF,None: n_random_points= (lenthg ts)//2 nb_inst_...
freq,720,1105,85


In [110]:
pd.pivot_table(df_result, values='time', index=['dataset_name'], aggfunc='sum')/(60*60)

,time
dataset_name,
BirdChicken,14.590090
Chinatown,0.038223
Coffee,3.435429
DistalPhalanxOutlineAgeGroup,19.866818
DistalPhalanxOutlineCorrect,38.246154
GunPoint,6.767564
ItalyPowerDemand,0.202361
ShapeletSim,11.955152
SmoothSubspace,0.395690


In [111]:
df_result['time'].sum()/(60*60)

114.51905549493101

#### Generate CD Diagram Tunning Hyperparameter

In [112]:
#create dataset to generate cd diagram
df_perf=pd.pivot_table(df_result, values='accuracy', index=['classifier_name','dataset_name'], aggfunc='mean')
df_perf=df_perf.reset_index()
#df_perf.classifier_name.unique()


In [113]:
param_per_ds=pd.pivot_table(df_perf, index=['dataset_name'],values=['classifier_name'] ,aggfunc='count')
param_per_ds=param_per_ds.reset_index()
complete_ds=param_per_ds[param_per_ds.classifier_name==max(param_per_ds.classifier_name)]["dataset_name"].unique()

df_perf=df_perf[df_perf.dataset_name.isin(complete_ds)]
df_perf.dataset_name.unique()

array(['Chinatown', 'Coffee', 'DistalPhalanxOutlineCorrect', 'GunPoint',
       'ItalyPowerDemand', 'SmoothSubspace', 'SonyAIBORobotSurface2'],
      dtype=object)

In [114]:
from cd_function import *
#generate hyperparameter tuning cd diagrams
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^ACF&PACF:')], labels=True, title="ACF&PACF comparison", name_file='images_cd_diagram/cd-diagram_ACF&PACF.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^PACF:')], labels=True, title="PACF comparison", name_file='images_cd_diagram/cd-diagram_PACF.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^ACF:')], labels=True, title="ACF comparison", name_file='images_cd_diagram/cd-diagram_ACF.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^None:')], labels=True, title="None comparison", name_file='images_cd_diagram/cd-diagram_None.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^Max ACF:')], labels=True, title="Max ACF comparison", name_file='images_cd_diagram/cd-diagram_MaxACF.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^Max PACF:')], labels=True, title="Max PACF comparison", name_file='images_cd_diagram/cd-diagram_MaxPACF.png')
draw_cd_diagram(df_perf, labels=True, title="overall Hyperparameter Tunning", name_file='images_cd_diagram/cd-diagram_overall_tunning.png')



/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3428: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")
/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3428: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small 

In [115]:
_, average_ranks, _ = wilcoxon_holm(df_perf=df_perf)
min_rank= min(average_ranks)
average_ranks=pd.DataFrame(average_ranks)
average_ranks

/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3428: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


,0
ACF: n_random_points=1 nb_inst_per_class=1,194.000000
Max ACF: n_random_points=1 nb_inst_per_class=1,193.785714
Max PACF: n_random_points=1 nb_inst_per_class=1,188.785714
Max ACF: n_random_points=10 nb_inst_per_class=1,185.000000
ACF: n_random_points=100 nb_inst_per_class=1,184.142857
...,...
PACF: n_random_points= (lenthg ts)//2 nb_inst_per_class=100,45.428571
ACF&PACF: n_random_points=50 nb_inst_per_class=100,44.714286
ACF&PACF: n_random_points= (lenthg ts)//2 nb_inst_per_class=30,44.214286
ACF&PACF: n_random_points= (lenthg ts)//2 nb_inst_per_class=50,42.500000


In [116]:
#generate example cd diagrams
#draw_cd_diagram(pd.read_csv('example_cd_diagram.csv'), labels=True, title="Example CD Diagram", name_file='cd-diagram_example.png')

#### Focus on most accurate lenght method

##### Overall Accuracy

In [117]:
# highest accurate hyperparameters
average_ranks[average_ranks[0]==min_rank][0].index

Index(['ACF&PACF: n_random_points= (lenthg ts)//2 nb_inst_per_class=100'], dtype='object')

In [118]:
# filter by method with ACF&PACF
# create a pivot table with the mean of score by hyperparameter
df_result_acc=df_result[df_result["len_method"]=="ACF&PACF"]
pd.pivot_table(df_result_acc, values='accuracy', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
rpoint,,,,,,
(lenthg ts)//2,0.927406,0.861396,0.919322,0.937549,0.934906,0.938619
1,0.909112,0.714988,0.696115,0.789771,0.790266,0.785953
10,0.924257,0.727182,0.790894,0.796666,0.795204,0.790671
100,0.925995,0.777466,0.786825,0.792186,0.795236,0.794422
30,0.921517,0.746645,0.782546,0.795352,0.793410,0.793275
50,0.925417,0.784883,0.787817,0.801271,0.777855,0.797541


In [119]:
# create a pivot table with the variatioon of score by hyperparameter
pd.pivot_table(df_result_acc, values='accuracy', index=['rpoint'],columns=['nb_per_class'], aggfunc='var')

nb_per_class,(max instances per class)//2,1,10,100,30,50
rpoint,,,,,,
(lenthg ts)//2,0.004555,0.009777,0.005628,0.003925,0.004491,0.004211
1,0.005565,0.062568,0.077627,0.040615,0.037238,0.046755
10,0.005050,0.059836,0.040519,0.047997,0.045891,0.050160
100,0.005213,0.035086,0.047333,0.052971,0.049867,0.051297
30,0.005852,0.046467,0.055026,0.048539,0.045650,0.051740
50,0.004531,0.031979,0.044719,0.047246,0.064878,0.051205


##### Overall time complexity

In [120]:
# create a pivot table with the mean of time spent in minutes
pd.pivot_table(df_result_acc, values=['time','cweights_time','fsubsequence_time','tdataset_time','tclassifier_time'], index=['dataset_name'], aggfunc='mean')/60

,cweights_time,fsubsequence_time,tclassifier_time,tdataset_time,time
dataset_name,,,,,
BirdChicken,0.001775,1.845793,0.000515,3.106501,4.954678
Chinatown,0.000153,0.001468,0.000136,0.004364,0.006146
Coffee,0.001170,0.102598,0.000719,2.367384,2.472001
DistalPhalanxOutlineAgeGroup,0.000686,0.038889,0.010279,6.936574,6.986530
DistalPhalanxOutlineCorrect,0.000789,0.029408,0.014176,11.280322,11.324899
GunPoint,0.000702,0.035006,0.001057,1.556273,1.593144
ItalyPowerDemand,0.000133,0.002412,0.000269,0.024644,0.027488
ShapeletSim,0.002136,2.069743,0.000366,1.631211,3.703531
SmoothSubspace,0.000103,0.002886,0.000560,0.040557,0.044144


In [121]:
# create a pivot table with the mean of time spent in minutes by random points and instances per class
pd.pivot_table(df_result_acc, values='time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')/60

nb_per_class,(max instances per class)//2,1,10,100,30,50
rpoint,,,,,,
(lenthg ts)//2,2.550256,0.070278,0.747069,2.446032,1.534642,1.797074
1,0.077047,0.056340,0.424776,0.516456,0.507142,0.507167
10,0.573692,0.078892,0.720324,1.747770,1.123094,1.364348
100,5.550490,0.375743,3.697348,14.121683,6.960061,10.042684
30,1.671064,0.140927,1.359760,4.634129,2.447615,3.326647
50,2.808909,0.218616,2.068863,7.552542,4.242656,5.094940


##### Calculate weights time complexity

In [122]:
pd.pivot_table(df_result_acc, values='cweights_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
rpoint,,,,,,
(lenthg ts)//2,0.019900,0.020351,0.019722,0.020405,0.018777,0.020023
1,0.019432,0.039505,0.034749,0.043230,0.037325,0.038426
10,0.018460,0.041367,0.036154,0.046714,0.043347,0.042696
100,0.018009,0.052451,0.043583,0.046503,0.049913,0.050062
30,0.019852,0.044675,0.042425,0.049988,0.046137,0.050697
50,0.021580,0.046173,0.046191,0.052046,0.046225,0.050087


##### Finding subsequences time complexity

In [123]:
pd.pivot_table(df_result_acc, values='fsubsequence_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
rpoint,,,,,,
(lenthg ts)//2,0.640824,0.066150,0.583863,0.988845,0.997901,0.917418
1,0.693961,2.466725,23.405273,22.474001,26.292221,24.590836
10,0.720214,2.626947,23.550564,22.773997,26.564345,25.116368
100,0.662200,2.853488,27.140589,27.841491,25.001743,38.272257
30,0.636098,2.396395,22.864531,28.764379,27.377542,29.920977
50,0.598987,2.856334,28.456786,26.645855,30.253524,26.800147


##### Transform Dataset time complexity

In [124]:
pd.pivot_table(df_result_acc, values='tdataset_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
rpoint,,,,,,
(lenthg ts)//2,152.227000,4.109572,44.180025,145.638523,90.987797,106.796738
1,3.889762,0.851994,1.973064,8.379904,4.031182,5.722793
10,33.632106,2.011591,19.536340,81.891388,40.658658,56.582710
100,332.139485,19.530270,194.459079,817.589168,391.597881,562.884826
30,99.525418,5.938078,58.572563,247.579255,119.253728,169.298435
50,167.779839,10.107716,95.477171,408.686579,198.648432,278.487083


##### Classifier time complexity

In [125]:
pd.pivot_table(df_result_acc, values='tclassifier_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
rpoint,,,,,,
(lenthg ts)//2,0.119548,0.019378,0.037166,0.102260,0.067066,0.082490
1,0.018818,0.019162,0.072361,0.088792,0.066804,0.076676
10,0.048302,0.052410,0.095189,0.151160,0.117306,0.116650
100,0.202596,0.104079,0.191388,1.807877,0.944107,1.341054
30,0.078398,0.074968,0.103886,1.646955,0.174793,0.323106
50,0.129153,0.103675,0.147375,17.759222,25.605202,0.350316
